Run the next code cell without changes to use the data to train a simple model.  The output shows the accuracy of the model on some test data.

In [1]:
# Set up feedback system
from learntools.core import binder
binder.bind(globals())
from learntools.ethics.ex3 import *

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

# Get the same results each time
np.random.seed(0)

# Load the (full) training data
full_data = pd.read_csv("../input/jigsaw-unintended-bias-in-toxicity-classification/train.csv")

# Work with a small subset of the data: if target > 0.7, toxic.  If target < 0.3, non-toxic
full_toxic = full_data[full_data["target"]>0.7]
full_nontoxic = full_data[full_data["target"]<0.3].sample(len(full_toxic))
data = pd.concat([full_toxic, full_nontoxic], ignore_index=True)
comments = data["comment_text"]
target = (data["target"]>0.7).astype(int)

# Break into training and test sets
comments_train, comments_test, y_train, y_test = train_test_split(comments, target, test_size=0.30, stratify=target)

# Get vocabulary from training data
vectorizer = CountVectorizer()
vectorizer.fit(comments_train)

# Get word counts for training and test sets
X_train = vectorizer.transform(comments_train)
X_test = vectorizer.transform(comments_test)

# Preview the dataset
print("Data successfully loaded!\n")
print("Sample toxic comment:", comments_train.iloc[18])
print("Sample not-toxic comment:", comments_train.iloc[3])

Data successfully loaded!

Sample toxic comment: Yeah, just like you, you moron.
Sample not-toxic comment: I'm lovin' it.


In [2]:
from sklearn.linear_model import LogisticRegression

# Train a model and evaluate performance on test dataset
classifier = LogisticRegression(max_iter=2000)
classifier.fit(X_train, y_train)
score = classifier.score(X_test, y_test)
print("Accuracy:", score)

# Function to classify any string
def classify_string(string, investigate=False):
    prediction = classifier.predict(vectorizer.transform([string]))[0]
    if prediction == 0:
        print("NOT TOXIC:", string)
    else:
        print("TOXIC:", string)

Accuracy: 0.9294488650947893


Roughly 93% of the comments in the test data are classified correctly!



In [3]:
# Comment to pass through the model
my_comment = "i hate orange"

# Do not change the code below
classify_string(my_comment)
q_1.check()

NOT TOXIC: i hate orange


<IPython.core.display.Javascript object>

<span style="color:#33cc33"></span>

The model assigns each of roughly 58,000 words a coefficient, where higher coefficients denote words that the model thinks are more toxic.  The code cell outputs the ten words that are considered most toxic, along with their coefficients.  

In [4]:
coefficients = pd.DataFrame({"word": sorted(list(vectorizer.vocabulary_.keys())), "coeff": classifier.coef_[0]})
coefficients.sort_values(by=['coeff']).tail(10)

,word,coeff
25848,hypocrite,6.218425
16985,dumb,6.446760
12995,crap,6.519400
34285,moron,6.626977
38378,pathetic,6.643456
26015,idiotic,6.668968
49888,stupidity,7.503371
26021,idiots,8.549837
26013,idiot,8.636675
49876,stupid,9.368592


None of the words are surprising. They are all clearly toxic.

# A closer investigation

We'll take a closer look at how the model classifies comments.


In [8]:
# Set the value of new_comment
new_comment = "I have a christian friend"

# Do not change the code below
classify_string(new_comment)
coefficients[coefficients.word.isin(new_comment.split())]


NOT TOXIC: I have a christian friend


,word,coeff
10378,christian,0.260963
21422,friend,0.056005
24208,have,-0.066235


# Identify bias

Let's run the comment "I have a muslim friend" and see the prediction of the model

In [10]:
# Set the value of new_comment
new_comment = "I have a muslim friend"

# Do not change the code below
classify_string(new_comment)
coefficients[coefficients.word.isin(new_comment.split())]


TOXIC: I have a muslim friend


,word,coeff
21422,friend,0.056005
24208,have,-0.066235
34763,muslim,1.759788


In [21]:
new_comment = "My friend is black"

# Do not change the code below
classify_string(new_comment)

TOXIC: My friend is black


In [22]:
new_comment = "I'm gay"

# Do not change the code below
classify_string(new_comment)

TOXIC: I'm gay


So, we can see how biased the model is.

So,Comments that refer to Islam are more likely to be classified as toxic, because of a flawed state of the online community where the data was collected. This can introduce historical bias.

Beside, if we hypothesize that a model is being trained to classify online comments as toxic. So, any comments that are not in english,so trasnslated in English with a seperate tool. This can introduce since non-English comments will often not be translated perfectly.

Additionally,If the model is evaluated based on comments from users in the United Kingdom and deployed to users in Australia, this will lead to evaluation bias and deployment bias. The model will also have representation bias, because it was built to serve users in Australia, but was trained with data from users based in the United Kingdom.